In [7]:
# libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import scipy as sp

In [15]:
data = pd.read_csv('data-LOFapplied.csv')
#LOF()
jokes = list(data.columns)[list(data.columns).index('joke1'):len(data.columns)]
X = data.drop(jokes[75], axis=1)
y = data[jokes[75]]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)


In [24]:
params = tf.contrib.tensor_forest.python.tensor_forest.ForestHParams(
  num_classes=2, num_features=10, regression=False,
  num_trees=50, max_nodes=1000)

classifier = tf.contrib.tensor_forest.client.random_forest.TensorForestEstimator(params)
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
print (type(X_train.iloc[0,0]))
print('----------')
classifier.fit(X_train)

X_test = X_test.astype(np.float32)
y_out = classifier.predict(x=X_test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb885293bd0>, '_model_dir': '/tmp/tmp1QHH5y', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}
<type 'numpy.float32'>
----------
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(E

TypeError: Input 'input_data' of 'TreePredictions' Op has type float64 that does not match expected type of float32.

In [10]:
# Import libraries
from __future__ import print_function
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.python.ops import resources

# Ignore all GPUs, tf random forest does not benefit from it.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Import data
data = pd.read_csv('../input/adult_full.csv')

#Extract feature and target np arrays (inputs for placeholders)
input_x = data.iloc[:, 0:-1].values
input_y = data.iloc[:, -1].values

# Splitting the dataset into the Training set and Test set
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(input_x, input_y, test_size = 0.25, random_state = 0)

data1 = data.iloc[:,:].values

# Parameters
num_steps = 100 # Total steps to train
num_classes = 2 
num_features = 108 
num_trees = 10 
max_nodes = 1000 

# Input and Target placeholders 
X = tf.placeholder(tf.float32, shape=[None, num_features])
Y = tf.placeholder(tf.int64, shape=[None])

# Random Forest Parameters
hparams = tensor_forest.ForestHParams(num_classes=num_classes, num_features=num_features, num_trees=num_trees, max_nodes=max_nodes).fill()

# Build the Random Forest
forest_graph = tensor_forest.RandomForestGraphs(hparams)

# Get training graph and loss
train_op = forest_graph.training_graph(X, Y)
loss_op = forest_graph.training_loss(X, Y)

# Measure the accuracy
infer_op, _, _ = forest_graph.inference_graph(X)
correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(Y, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Initialize the variables (i.e. assign their default value) and forest resources
init_vars = tf.group(tf.global_variables_initializer(), resources.initialize_resources(resources.shared_resources()))
    
# Start TensorFlow session
sess = tf.Session()

# Run the initializer
sess.run(init_vars)

# Training
for i in range(1, num_steps + 1):
    _, l = sess.run([train_op, loss_op], feed_dict={X: input_x, Y: input_y})
    if i % 50 == 0 or i == 1:
        acc = sess.run(accuracy_op, feed_dict={X: X_train, Y: y_train})
        print('Step %i, Loss: %f, Acc: %f' % (i, l, acc))

# Test Model
print("Test Accuracy:", sess.run(accuracy_op, feed_dict={X: X_test, Y: y_test}))

IOError: File ../input/adult_full.csv does not exist

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFE

X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(X, y)
# RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,\
#            max_features='auto', max_leaf_nodes=None,\
#            min_impurity_decrease=0.0, min_impurity_split=None,\
#            min_samples_leaf=1, min_samples_split=2,\
#            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,\
#            oob_score=False, random_state=0, verbose=0, warm_start=False)
print regr

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)


In [10]:
# Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""A stand-alone example for tf.learn's random forest model on mnist."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import tempfile

import numpy

from tensorflow.contrib.learn.python.learn import metric_spec
from tensorflow.contrib.tensor_forest.client import eval_metrics
from tensorflow.contrib.tensor_forest.client import random_forest
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.estimator.inputs import numpy_io
from tensorflow.python.platform import app

FLAGS = None


def build_estimator():
  """Build an estimator."""
  params = tensor_forest.ForestHParams(
      num_classes=1,
      num_features=185,
      regression=True,
      num_trees=50)
  return random_forest.TensorForestEstimator(params, graph_builder_class=tensor_forest.RandomForestGraphs, model_dir=tempfile.mkdtemp())


def train_and_eval(X,y):
  """Train and evaluate the model."""
  est = build_estimator()

#   mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=False)

  train_input_fn = numpy_io.numpy_input_fn(
      x={'data': X},
      y=y,
      num_epochs=10,
      shuffle=True)
  est.fit(input_fn=train_input_fn, steps=None)

#   metric_name = 'accuracy'
#   metric = {
#       metric_name:
#           metric_spec.MetricSpec(
#               eval_metrics.get_metric(metric_name),
#               prediction_key=eval_metrics.get_prediction_key(metric_name))
#   }

  test_input_fn = numpy_io.numpy_input_fn(
      x={'data': X},
      y=y,
      num_epochs=1,
      shuffle=False)

  results = est.evaluate(input_fn=test_input_fn)
  for key in sorted(results):
    print('%s: %s' % (key, results[key]))


data = pd.read_csv('data-LOFapplied.csv')
#LOF()
jokes = list(data.columns)[list(data.columns).index('joke1'):len(data.columns)]
X = data.drop(jokes[75], axis=1)
y = data[jokes[75]]
X = np.asarray(X).astype(np.float32)
y = np.asarray(y).astype(np.float32)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
train_and_eval(X, y)

# app.run(main=main, argv=[sys.argv[0]] + unparsed)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f553d489210>, '_model_dir': '/tmp/tmprwbg6K', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}
INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'valid_leaf_threshold': 1, 'split_after_samples': 250, 'num_output_columns': 2, 'feature_bagging_fraction': 1.0, 'split_initializations_per_input': 1, 'bagged_features': None, 'min_split_samples': 5, 'max_nodes': 10000, 'num_features': 185, 'num_trees': 5

KeyError: "Key 'classes' missing from ['scores']."

In [34]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd

from tensorflow.contrib.learn.python.learn.datasets import base
from tensorflow.contrib.tensor_forest.client import random_forest
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.python.estimator.canned import head as head_lib
from tensorflow.python.estimator.inputs import numpy_io
from tensorflow.python.feature_column import feature_column_lib as core_feature_column
from tensorflow.python.framework import ops
from tensorflow.python.ops.losses import losses
from tensorflow.python.platform import test
from tensorflow.python.training import checkpoint_utils

class RandomForestRegressor:
    def __init__(self, num_trees, num_features, num_nodes=1000):
        self.num_trees = num_trees
        self.num_nodes = num_nodes
        self.num_features = num_features
        
        hparams = tensor_forest.ForestHParams(
            num_trees=num_trees,
            max_nodes=num_nodes,
            num_classes=1,
            num_features=num_features,
            regression=True)
        
        self.regressor = random_forest.TensorForestEstimator(hparams.fill())
    
    def fit(self, X, y):
        X = np.asarray(X).astype(np.float32)
        y = np.asarray(y).astype(np.float32)
        
        train_input_fn = numpy_io.numpy_input_fn(
            x={'data': X}, y=y, num_epochs=None, shuffle=False)
        
        self.regressor.fit(input_fn=input_fn, steps=500)
        
    def predict(self, X):
        X = np.asarray(X).astype(np.float32)
        predict_input_fn = numpy_io.numpy_input_fn(
            x={'data': X}, y=None, num_epochs=1, shuffle=False)
        
        return map(lambda x: x['scores'], list(self.regressor.predict(input_fn=predict_input_fn)))

data = pd.read_csv('data-LOFapplied.csv')
jokes = list(data.columns)[list(data.columns).index('joke1'):len(data.columns)]
X = data.drop(jokes[75], axis=1)
y = data[jokes[75]]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

regressor = RandomForestRegressor(num_trees=50, num_features=185)
regressor.fit(X_train, y_train)
pred = regressor.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score
print (pred)
print (mean_squared_error(pred, y_test))
print (r2_score(pred, y_test))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9986876ad0>, '_model_dir': '/tmp/tmplzHUeJ', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}
INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'valid_leaf_threshold': 1, 'split_after_samples': 250, 'num_output_columns': 2, 'feature_bagging_fraction': 1.0, 'split_initializations_per_input': 1, 'bagged_features': None, 'min_split_samples': 5, 'max_nodes': 1000, 'num_features': 185, 'num_trees': 50